In [1]:
"""
Deposit File Creation
"""
from pathlib import Path
from datetime import datetime
from dateutil.relativedelta import relativedelta

import pandas as pd # type: ignore

import src.deposit_file.create_deposit_dataset

# def deposit_dataset_execution():
data = src.deposit_file.create_deposit_dataset.fetch_data()

# Unpack data
acctcommon = data['acctcommon'].copy()
acctstatistichist = data['acctstatistichist'].copy()
wh_deposits = data['wh_deposit'].copy()
historical_acctcommon = data['historical_acctcommon'].copy()
househldacct = data['househldacct']



In [2]:

three_month_df = src.deposit_file.create_deposit_dataset.filter_on_trailing_months(historical_acctcommon, 3)

In [4]:
three_month_df

,acctnbr,effdate,monthendyn,notemtdavgbal
815129,151111946,2025-06-30,Y,2900.70
773293,29012465,2025-06-30,Y,65590.25
773291,29012252,2025-06-30,Y,100.00
773290,29011825,2025-06-30,Y,57.16
773289,29011620,2025-06-30,Y,71.50
...,...,...,...,...
668652,4465041551,2025-04-30,Y,150265.81
668651,4465023782,2025-04-30,Y,39909.07
668650,4465038003,2025-04-30,Y,23.64
668649,4465086717,2025-04-30,Y,831.57


In [3]:

# three_month_df.to_excel(Path('ThreeMonthCheck.xlsx',index=False))
twelve_month_df = src.deposit_file.create_deposit_dataset.filter_on_trailing_months(historical_acctcommon, 12)

In [5]:

# twelve_month_df.to_excel(Path('TwelveMonthCheck.xlsx',index=False))

# year_ago_bal = src.deposit_file.create_deposit_dataset.get_year_ago_bal(historical_acctcommon)
# Example usage

three_month_avg = src.deposit_file.create_deposit_dataset.average_balance_over_period(three_month_df, '3Mo_AvgBal')
twelve_month_avg = src.deposit_file.create_deposit_dataset.average_balance_over_period(twelve_month_df, 'TTM_AvgBal')



In [6]:
three_month_avg

,acctnbr,3Mo_AvgBal
0,100032,10107.766667
1,100036,1696.910000
2,100038,7184.946667
3,100042,8735.613333
4,100047,10163.123333
...,...,...
63987,5526703870,3070.680000
63988,5528026012,614.133333
63989,5528043731,1535.340000
63990,5528143469,307.066667


In [7]:
# Current date
current_date = datetime.now()

# TTM: Trailing Twelve Months
start_date_ttm = (current_date - relativedelta(years=1)).replace(day=1, hour=0, minute=0, second=0, microsecond=0)
end_date_ttm = current_date
dod_ttm, nsf_ttm = src.deposit_file.create_deposit_dataset.filter_acctstatistic(acctstatistichist, start_date_ttm, end_date_ttm, prefix="TTM")

In [9]:
start_date_ttm

datetime.datetime(2024, 7, 1, 0, 0)

In [10]:
end_date_ttm

datetime.datetime(2025, 7, 14, 13, 57, 57, 415473)

In [8]:
dod_ttm

,acctnbr,TTM_DAYS_OVERDRAWN
0,100696,89.0
1,100709,34.0
2,100720,1.0
3,100770,1.0
4,100819,2.0
...,...,...
5933,4465593295,60.0
5934,4465634668,14.0
5935,4465673248,40.0
5936,4465674155,4.0


In [ ]:
# Current date
current_date = datetime.now()

# TTM: Trailing Twelve Months
start_date_ttm = (current_date - relativedelta(years=1)).replace(day=1, hour=0, minute=0, second=0, microsecond=0)
end_date_ttm = current_date
dod_ttm, nsf_ttm = src.deposit_file.create_deposit_dataset.filter_acctstatistic(acctstatistichist, start_date_ttm, end_date_ttm, prefix="TTM")

# YTD: Year-to-Date
start_date_ytd = datetime(current_date.year, 1, 1)
end_date_ytd = current_date
dod_ytd, nsf_ytd = src.deposit_file.create_deposit_dataset.filter_acctstatistic(acctstatistichist, start_date_ytd, end_date_ytd, prefix="YTD")


df = src.deposit_file.create_deposit_dataset.quality_control_and_merging(
    acctcommon, 
    wh_deposits, 
    three_month_avg, 
    twelve_month_avg, 
    year_ago_bal, 
    dod_ttm, 
    nsf_ttm,
    dod_ytd,
    nsf_ytd
)

df = src.deposit_file.create_deposit_dataset.add_householdnbr(df, househldacct)

# Match to existing daily deposit (but there are more columns here that could be used)
rename_mapping = {
    "mjaccttypcd":"MAJOR",
    "currmiaccttypcd":"MINOR",
    "product":"PRODUCT",
    "curracctstatcd":"STATUS",
    "acctnbr":"ACCTNBR",
    "ownersortname":"OWNERNAME",
    "acctofficer":"ACCTOFFICER",
    "notebal":"NOTEBAL",
    "TTM_AvgBal":"TTM Average Balance",
    "TTM_DAYS_OVERDRAWN":"TTM Overdrafts",
    "ytdavgbal":"YTDAVGBAL",
    "YTD_DAYS_OVERDRAWN":"YTD Overdrafts",
    "effdate":"EFFDATE",
    "contractdate":"CONTRACTDATE",
    "householdnbr":"HOUSEHOLDNBR"

}

desired_order = [
    "MAJOR",
    "MINOR",
    "PRODUCT",
    "STATUS",
    "ACCTNBR",
    "OWNERNAME",
    "ACCTOFFICER",
    "NOTEBAL",
    "TTM Average Balance",
    "TTM Overdrafts",
    "YTDAVGBAL",
    "YTD Overdrafts",
    "HOUSEHOLDNBR",
    "EFFDATE",
    "CONTRACTDATE"
]

dropin_deposit_df = df.rename(columns=rename_mapping).loc[:,desired_order].copy()

# return df, dropin_deposit_df
